In [85]:
!pip install groq


In [86]:
from getpass import getpass
import os

os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")


Enter your Groq API key: ··········


In [87]:
from groq import Groq
import os

client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [88]:
import json

def classify_complaint(text):
    prompt = f"""
You are a complaint classification system.

Classify the complaint into exactly one of:
- food_spoiled
- food_tampered
- late_delivery
- wrong_item

Return JSON only:
{{
  "category": "",
  "confidence": 0.0
}}

Complaint:
{text}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a strict JSON-only classification engine."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    raw_output = response.choices[0].message.content

    try:
        return json.loads(raw_output)
    except:
        return {"error": "Invalid JSON", "raw": raw_output}


In [89]:
result = classify_complaint("The food quality was not good")
print(result)
print(type(result))


{'category': 'food_spoiled', 'confidence': 0.8}
<class 'dict'>


In [90]:
import json

policies = [
  {
    "id": "food_spoiled",
    "title": "Spoiled Food Policy",
    "content": "If a customer reports spoiled food, apologize sincerely. Ask for image proof. Offer a full refund. If the same customer reports spoiled food more than twice in 30 days, escalate to human support."
  },
  {
    "id": "food_tampered",
    "title": "Tampered Food Policy",
    "content": "If food appears tampered, apologize immediately. Offer full refund without delay. Escalate to safety team. Mark delivery partner for investigation."
  },
  {
    "id": "late_delivery",
    "title": "Late Delivery Policy",
    "content": "If delivery is delayed more than 45 minutes beyond estimated time, offer 30 percent refund. If more than 90 minutes, offer full refund."
  },
  {
    "id": "wrong_item",
    "title": "Wrong Item Policy",
    "content": "If a customer receives the wrong item, wrong meal, incorrect order, or missing item, apologize sincerely. Offer replacement or full refund. Ask whether they prefer refund or redelivery. Escalate if repeated more than twice in 30 days."
  }
]

with open("policies.json", "w") as f:
    json.dump(policies, f, indent=2)


In [91]:
with open("policies.json", "r") as f:
    policies = json.load(f)


In [92]:
def retrieve_policy(category):
    for policy in policies:
        if policy["id"] == category:
            return policy["content"]
    return "No policy found."


In [93]:
#Test1
result = classify_complaint("My order arrived 1 hour late")
policy_text = retrieve_policy(result["category"])

print("Classification:", result)
print("Policy:", policy_text)


Classification: {'category': 'late_delivery', 'confidence': 1.0}
Policy: If delivery is delayed more than 45 minutes beyond estimated time, offer 30 percent refund. If more than 90 minutes, offer full refund.


In [94]:
import json

def generate_response(complaint, category, policy_text):
    prompt = f"""
You are a professional customer support assistant.

You must strictly follow the company policy provided below.

Company Policy:
{policy_text}

Customer Complaint:
{complaint}

Instructions:
- Follow policy strictly.
- Do not invent compensation beyond policy.
- Return JSON only:

{{
  "response_text": "",
  "action_taken": "",
  "escalate": false
}}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a strict JSON-only customer support AI."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    raw_output = response.choices[0].message.content

    try:
        return json.loads(raw_output)
    except:
        return {"error": "Invalid JSON", "raw": raw_output}


In [95]:
def process_complaint(complaint):
    classification = classify_complaint(complaint)

    if classification.get("confidence", 0) < 0.6:
        return {
            "response_text": "Your complaint requires human review.",
            "action_taken": "Escalated to human support",
            "escalate": True
        }

    policy_text = retrieve_policy(classification["category"])

    response = generate_response(
        complaint,
        classification["category"],
        policy_text
    )

    return response


In [96]:
#Test2
complaint = "My order arrived 1 hour late"

result = classify_complaint(complaint)
policy_text = retrieve_policy(result["category"])

final_response = generate_response(
    complaint,
    result["category"],
    policy_text
)

print(final_response)


{'response_text': 'We apologize for the delay in your order. Since your order arrived 1 hour late, which is more than 45 minutes beyond the estimated time, we are offering a 30% refund.', 'action_taken': 'Refund of 30% will be processed', 'escalate': False}


In [97]:
#Test3
print(process_complaint("Service was terrible"))


{'response_text': 'Your complaint requires human review.', 'action_taken': 'Escalated to human support', 'escalate': True}


In [98]:
!pip install sentence-transformers faiss-cpu


In [99]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedding_model = SentenceTransformer("all-mpnet-base-v2")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [100]:
policy_embeddings = embedding_model.encode(policy_texts, normalize_embeddings=True)

dimension = policy_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)   # NOT L2
index.add(np.array(policy_embeddings))


In [101]:
def retrieve_policy_rag(query, top_k=1):
    query_embedding = embedding_model.encode([query], normalize_embeddings=True)
    distances, indices = index.search(np.array(query_embedding), top_k)

    results = []
    for idx in indices[0]:
        results.append({
            "policy_id": policies[idx]["id"],
            "content": policies[idx]["content"]
        })

    return results


In [102]:
print(retrieve_policy_rag("My food smelled bad and looked spoiled"))
print(retrieve_policy_rag("Delivery was very late"))
print(retrieve_policy_rag("I received the wrong meal"))


[{'policy_id': 'food_spoiled', 'content': 'If a customer reports spoiled food, apologize sincerely. Ask for image proof. Offer a full refund. If the same customer reports spoiled food more than twice in 30 days, escalate to human support.'}]
[{'policy_id': 'late_delivery', 'content': 'If delivery is delayed more than 45 minutes beyond estimated time, offer 30 percent refund. If more than 90 minutes, offer full refund.'}]
[{'policy_id': 'wrong_item', 'content': 'If a customer receives the wrong item, wrong meal, incorrect order, or missing item, apologize sincerely. Offer replacement or full refund. Ask whether they prefer refund or redelivery. Escalate if repeated more than twice in 30 days.'}]


In [103]:
print(retrieve_policy_rag("I received the wrong meal"))


[{'policy_id': 'wrong_item', 'content': 'If a customer receives the wrong item, wrong meal, incorrect order, or missing item, apologize sincerely. Offer replacement or full refund. Ask whether they prefer refund or redelivery. Escalate if repeated more than twice in 30 days.'}]
